## Submitting a job with the [`otello`](https://github.com/hysds/otello) python library

Once your job-type has been registered and built (see: [pge_create.ipynb](pge_create.ipynb)), jobs can be submitted from python using the steps laid out in this notebook.

While this notebook only shows submission of a single job/parameter-set, you can map or iterate over a collection of input parameter sets to efficiently submit large batches of jobs.

### Establish an otello `Mozart` instance to communicate with the HySDS cluster controller.
It will be necessary to provide credentials the first time you initialise otello.

When prompted for the HySDS host, include the protocol, e.g. https://my-mozart.jpl.nasa.gov

When prompted for "HySDS cluster authenticated", enter 'y' if the cluster requires a password to access.

In [ ]:
import os
from pathlib import Path
from pprint import pprint
import otello

if not os.path.exists(f"{Path.home()}/.config/otello/config.yml"):
    otello.client.initialize()

m = otello.mozart.Mozart()

#### Instantiating a `JobType` object
You will need to ensure that the the tag (i.e. the portion following the colon) matches the branch name used by your repository.

You will also need to replace `<YOUR_JOB_TYPE_NAME>` with the name of your job's notebook (without the filetype extension).
e.g. if your notebook is hello_world_sample_pge.ipynb, the hysds-io.json and job-spec.jsons will have suffix `hello_world_sample_pge` and the value of `job_type` below would be `job-hello_world_sample_pge:develop` or similar.

In [ ]:
job_type='job-<YOUR_JOB_TYPE_NAME>:develop'  # This will need to be customised using the relevant action name (ie job/hysds-spec suffix) and branch name.

hw_job_def = m.get_job_types()[job_type]
hw_job_def.initialize()

### Getting useful information about the job-type

These commands list (respectively)
- the available queues
- the input schema (parameters) of the job-type
- the default arguments for the job-type

In [ ]:
hw_job_def.get_queues()
print(hw_job_def.describe())
pprint(hw_job_def.get_input_params())

### Specifying arguments to pass when running the job

Here is where user-defined parameter values are specified, making sure to remain consistent with value types as indicated in the *helloworld_notebook*. Default values are used where none are provided (as we've done here with *start_orbit_number*). *set_input_params* is called to pass the parameter values to the job.

In [ ]:
custom_parameters = {
    'str_arg' : 'THIS IS JOB 1'
}
    
hw_job_def.set_input_params(custom_parameters)
pprint(hw_job_def.get_input_params())

### Submitting the job
A job tag (useful for finding the job later) and job queue are specified. Both are optional. Job submission is asynchronous, so this call will return almost immediately.

In [ ]:
from datetime import datetime

sample_job_tag = f'{datetime.strftime(datetime.now(), "%Y%m%d")}_submit_job_test'
print(sample_job_tag)

job_run = hw_job_def.submit_job(tag=sample_job_tag, queue="factotum-job_worker-small")

### Determining job completion
Information about the job state will print periodically, until the job is completed.

In [ ]:
job_run.wait_for_completion()

### Submitting Multiple Jobs: Creating Input for Multiple Jobs
We can sumit multiple jobs which will run in parallel. First, lets create input parametrs for the jobs.

In [ ]:
input_params = [ {
    "tags" : "demo test 1",
    "params" : {
    'str_arg' : 'THIS IS JOB 1'
    }
  },
  {
    "tags" : "demo test 2",
    "params" : {
    'str_arg' : 'THIS IS JOB 2'
     }
  },
  {
    "tags" : "demo test 3",
    "params" : {
    'str_arg' : 'THIS IS JOB 3'
     }
  }
]

### Submitting Multiple Jobs: Submitting the Jobs 
Now submit the jobs and keep jobs in otello's job_set data structure

In [ ]:
job_set = otello.JobSet()

for i in range(len(input_params)):
    hw_job_def.set_input_params(input_params[i]["params"])
    job = hw_job_def.submit_job(tag=input_params[i]["tags"], queue="factotum-job_worker-small")
    job_set.append(job)

### Determining completion of all the jobs
Information about the job state will print periodically, until all the jobs are completed.

In [ ]:
job_set.wait_for_completion()

### Get information about the generated hello world products from the submitted jobs
#### The generated hello world products are stored in the cloud 

In [ ]:

prods = []
for job in job_set:
    try:
        prod = job.get_generated_products()
        print(json.dumps(prod, indent=2, sort_keys=True))
        prods.append(prod)
    except Exception as e:
        print(e)

### Download the generated hello world products from the cloud into this notebook
#### Here we use the AWS CLI to download the generated datasets.

In [ ]:
local_dirs = []
for prod in prods:
    try:
        prod_url = re.sub(r'^s3://.+?/(.+)$', r's3://\1', prod[0]["urls"][-1]) # get s3 url
        local_dir = os.path.basename(prod_url)
        if os.path.isdir(local_dir): shutil.rmtree(local_dir)
        !aws s3 sync $prod_url $local_dir
        local_dirs.append(local_dir)
    except Exception as e:
        print(e)
for local_dir in local_dirs:
    !ls $local_dir